In [5]:
import numpy as np
import operator

In [29]:
def file2matrix(filename):
    love_dictionary = {'largeDoses':3, 'smallDoses':2,'didntLike':1} # 三个类别(极具魅力的人, 魅力一般的人, 不喜欢的人)
    fr = open(filename) # 打开文件
    arrayOLines = fr.readlines() # 逐行打开
    
    numberOfLines = len(arrayOLines)  # 得到文件的行数
    returnMat = np.zeros((numberOfLines, 3)) # 初始化特征矩阵
    
    classLabelVector = []  # 初始化输出标签向量
    index = 0
    for line in arrayOLines:
        line = line.strip() # 删除字符串首尾空字符
        
        listFromLine = line.split('\t') #按照'\t'对字符串进行分割, listFromLine是列表
        # print(listFromLine)
        returnMat[index, :] = listFromLine[0:3] #listFromLine的0,1,2元素是特征, 赋值给returnMat的当前行
        
        if(listFromLine[-1].isdigit()): #如果listFormLine最后一个元素是数字
            classLabelVector.append(int(listFromLine[-1])) #直接赋值给classLabelVector
        else:
            classLabelVector.append(love_dictionary.get(listFromLine[-1])) # 根据字典love_dictionary转化为数字
        index += 1
        
    return returnMat, classLabelVector

In [33]:
def autoNorm(dataSet):
    minVals = dataSet.min(0)
    maxVals = dataSet.max(0)
    ranges = maxVals - minVals
    
    
    maxs = np.max(dataSet, axis=0)
    mins = np.min(dataSet, axis=0)
    new_data = (dataSet-mins)/(maxs-mins)
    #print(mins, maxs, '&&&&', new_data)
    normDataSet = np.zeros(np.shape(dataSet))
    m = dataSet.shape[0]
    normDataSet  = dataSet - np.tile(minVals,(m,1))
    normDataSet = normDataSet/np.tile(ranges,(m,1)) # normDataSet值被限定在[0, 1]之间
    #print(minVals, maxVals, '&&&&', normDataSet)
    
    return normDataSet, ranges, minVals

In [34]:
def classify0(inX, dataSet, labels, k): #inX是测试集, dataSet是训练集, labels是训练样本标签, k是取的最近邻个数
    dataSetSize = dataSet.shape[0] # 训练样本个数
    diffMat = np.tile(inX, (dataSetSize, 1)) - dataSet # np.tile:重复n此
    sqDiffMat = diffMat ** 2
    sqDistances = sqDiffMat.sum(axis=1)
    distances = sqDistances ** 0.5 # 上三步是计算欧氏距离
    sortedDistIndicies = distances.argsort() # 返回排序从小道大的索引位置
    classCount = {} #字典存储k近邻不同label出现的次数
    for i in range(k):
        voteTlabel = labels[sortedDistIndicies[i]]
        classCount[voteTlabel] = classCount.get(voteTlabel, 0) + 1 #对应label加1, classCount中若无此key,则默认为0
        
    sortedClassCount = sorted(classCount.items(), key = operator.itemgetter(1), reverse=True)
    return sortedClassCount[0][0]
    

In [35]:
def datingClassTest():
    hoRatio = 0.10      #整个数据集的10%用来测试
    datingDataMat, datingLabels = file2matrix('datingTestSet2.txt')       #导入数据集
    normMat, ranges, minVals = autoNorm(datingDataMat)    # 所有特征归一化
    m = normMat.shape[0]    # 样本个数
    numTestVecs = int(m*hoRatio)    # 测试样本个数
    errorCount = 0.0
    for i in range(numTestVecs):
        classifierResult = classify0(normMat[i, :], normMat[numTestVecs:m, :], datingLabels[numTestVecs:m], 3)
        print("the classifier came back with: %d, the real answer is: %d" % (classifierResult, datingLabels[i]))
        if (classifierResult != datingLabels[i]): errorCount += 1.0
    print("the total error rate is: %f" % (errorCount / float(numTestVecs)))    # 打印错误率
    print(errorCount) 

In [36]:
datingClassTest()

the classifier came back with: 3, the real answer is: 3
the classifier came back with: 2, the real answer is: 2
the classifier came back with: 1, the real answer is: 1
the classifier came back with: 1, the real answer is: 1
the classifier came back with: 1, the real answer is: 1
the classifier came back with: 1, the real answer is: 1
the classifier came back with: 3, the real answer is: 3
the classifier came back with: 3, the real answer is: 3
the classifier came back with: 1, the real answer is: 1
the classifier came back with: 3, the real answer is: 3
the classifier came back with: 1, the real answer is: 1
the classifier came back with: 1, the real answer is: 1
the classifier came back with: 2, the real answer is: 2
the classifier came back with: 1, the real answer is: 1
the classifier came back with: 1, the real answer is: 1
the classifier came back with: 1, the real answer is: 1
the classifier came back with: 1, the real answer is: 1
the classifier came back with: 1, the real answe

In [37]:
def classifyPerson():
    resultList = ['not at all', 'in small doses', 'in large doses']
    percentTats = float(input(\
                                  "percentage of time spent playing video games?"))
    ffMiles = float(input("frequent flier miles earned per year?"))
    iceCream = float(input("liters of ice cream consumed per year?"))
    datingDataMat, datingLabels = file2matrix('datingTestSet2.txt')
    normMat, ranges, minVals = autoNorm(datingDataMat)
    inArr = np.array([ffMiles, percentTats, iceCream, ])
    classifierResult = classify0((inArr - \
                                  minVals)/ranges, normMat, datingLabels, 3)
    print("You will probably like this person: %s" % resultList[classifierResult - 1])

In [39]:
classifyPerson()

percentage of time spent playing video games?50
frequent flier miles earned per year?1000
liters of ice cream consumed per year?5
You will probably like this person: in large doses
